# Objective:

Performance on FrcSub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/5329/democode/

In [ ]:
# import sys
# path = '/content/drive/MyDrive/5329/democode/'
# sys.path.append(path)

In [ ]:
import os
path = '/content/drive/MyDrive/5329/democode/'
os.chdir(path)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import torch.utils.data as Data
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from copy import deepcopy
import progressbar
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
import math
import scipy
import matplotlib.pyplot as plt
import collections
%matplotlib inline

USE_CUDA = torch.cuda.is_available()

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

if USE_CUDA:
    torch.cuda.manual_seed(1)


seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(seed)
gpu_available = torch.cuda.is_available()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#hyper parameters
NUM_EPOCHS = 100
MAX_RECORD_SIZE = 1e6
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
EMBEDDING_SIZE = 32 #knowledge_embedding_size, dimention of knowledge space

print(gpu_available)

In [ ]:
! /opt/bin/nvidia-smi

In [ ]:
def load_data(path, val_ratio, test_ratio) -> dict:
    full_data = pd.read_csv(path + 'data.txt', header=None, sep='\t').values.astype(np.int64) # CrossEntropy这里要改城int64
    knowledge_matrix = pd.read_csv(path + 'q.txt', header=None, sep='\t').values.astype(np.float32)
    students_num, items_num, skills_num = full_data.shape[0], full_data.shape[1], knowledge_matrix.shape[1]
    data = np.array([{'stu_id': stu_id, 'item_id': item_id, 'score': full_data[stu_id, item_id], 'knowledge': knowledge_matrix[item_id]}
          for stu_id in range(students_num) for item_id in range(items_num)])
    
    np.random.shuffle(data)
    
    train_val_data = data[ : int(len(data) * test_ratio)]
    test_data = data[int(len(data) * test_ratio) : ]
    
    train_data = train_val_data[ : int(len(train_val_data) * val_ratio)]
    val_data = train_val_data[int(len(train_val_data) * val_ratio) : ]
    
    return {'train_data': train_data, 'val_data': val_data, 'test_data': test_data, 'students_num': students_num, 'items_num': items_num, 'skills_num':  skills_num}

In [ ]:
FrcSub = load_data(path='./FrcSub/', val_ratio=0.8, test_ratio=0.8)

In [ ]:
print(FrcSub['train_data'].shape, FrcSub['val_data'].shape, FrcSub['test_data'].shape)
print(FrcSub['train_data'][1], '\n', FrcSub['val_data'][1], '\n', FrcSub['test_data'][1])

In [ ]:
train_data, val_data, test_data = FrcSub['train_data'], FrcSub['val_data'], FrcSub['test_data']
student_n, item_n, knowledge_n, knowledge_embed_size = FrcSub['students_num'], FrcSub['items_num'], FrcSub['skills_num'], EMBEDDING_SIZE

class MyDataset(Data.Dataset):
    def __init__(self, data):
        super(MyDataset, self).__init__() 
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]['stu_id'], self.data[idx]['item_id'], self.data[idx]['knowledge'], self.data[idx]['score']

train_dataset = MyDataset(train_data)
dataloader = Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [ ]:
for batch_stu_id, batch_item_id, batch_knowledge_id, batch_label in dataloader:
    print(batch_stu_id.dtype, batch_item_id.dtype, batch_label.dtype, batch_label.dtype)
    break

# NeuralCDM

In [ ]:
class NeuralCDM(nn.Module):
    '''
    NeuralCDM
    '''
    def __init__(self, student_n, exer_n, knowledge_n, knowledge_embed_size):
        self.knowledge_dim = knowledge_n
        self.exer_n = exer_n
        self.emb_num = student_n
        self.stu_dim = self.knowledge_dim
        self.prednet_input_len = self.knowledge_dim
        self.prednet_len1, self.prednet_len2 = 512, 256  # changeable
        
        self.knowledge_embed_size = knowledge_embed_size

        super(NeuralCDM, self).__init__()

        # network structure
        self.student_emb = nn.Embedding(self.emb_num, self.stu_dim) # (student_n, knowledge_n) -> (int , int)
        self.k_difficulty = nn.Embedding(self.exer_n, self.knowledge_dim)
        self.e_discrimination = nn.Embedding(self.exer_n, 1)
        self.prednet_full1 = nn.Linear(self.prednet_input_len, self.prednet_len1)
        self.drop_1 = nn.Dropout(p=0.5)
        self.prednet_full2 = nn.Linear(self.prednet_len1, self.prednet_len2)
        self.drop_2 = nn.Dropout(p=0.5)
        self.prednet_full3 = nn.Linear(self.prednet_len2, 1)

        # initialization
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)

    def forward(self, stu_id, exer_id, batch_knowledge_id, kn_emb):
        '''
        :param stu_id: LongTensor
        :param exer_id: LongTensor
        :param kn_emb: FloatTensor, the knowledge relevancy vectors
        :return: FloatTensor, the probabilities of answering correctly
        '''
        # before prednet
        stu_emb = torch.sigmoid(self.student_emb(stu_id))
        k_difficulty = torch.sigmoid(self.k_difficulty(exer_id))
        e_discrimination = torch.sigmoid(self.e_discrimination(exer_id)) * 10
        # prednet
        input_x = e_discrimination * (stu_emb - k_difficulty) * kn_emb
        input_x = self.drop_1(torch.sigmoid(self.prednet_full1(input_x)))
        input_x = self.drop_2(torch.sigmoid(self.prednet_full2(input_x)))
        output = torch.sigmoid(self.prednet_full3(input_x))

        return output

    def apply_clipper(self):
        clipper = NoneNegClipper()
        self.prednet_full1.apply(clipper)
        self.prednet_full2.apply(clipper)
        self.prednet_full3.apply(clipper)

    def get_knowledge_status(self, stu_id):
        stat_emb = torch.sigmoid(self.student_emb(stu_id))
        return stat_emb.data

    def get_exer_params(self, exer_id):
        k_difficulty = torch.sigmoid(self.k_difficulty(exer_id))
        e_discrimination = torch.sigmoid(self.e_discrimination(exer_id)) * 10
        return k_difficulty.data, e_discrimination.data
    
class NoneNegClipper(object):
    def __init__(self):
        super(NoneNegClipper, self).__init__()

    def __call__(self, module):
        if hasattr(module, 'weight'):
            w = module.weight.data
            a = torch.relu(torch.neg(w))
            w.add_(a)

In [ ]:
class AttentionLayer(nn.Module):
    
    def __init__(self, student_n, item_n, knowledge_n, knowledge_embed_size, n_heads=8):
        
        super(AttentionLayer, self).__init__()
        
        self.student_n = student_n
        self.item_n = item_n
        self.knowledge_n = knowledge_n
        self.knowledge_embed_size = knowledge_embed_size
        self.n_heads = n_heads
        self.d_model = self.knowledge_embed_size
        
        self.emb_stu = nn.Embedding(student_n, knowledge_embed_size) # Q
        self.emb_item = nn.Embedding(item_n, knowledge_embed_size) # K
        self.emb_knowledge = nn.Linear(knowledge_n, knowledge_embed_size) # V
        
        self.W_stu_knowledge = nn.Linear(self.d_model, knowledge_embed_size * self.n_heads, bias=False)
        
        self.W_item_knowledge = nn.Linear(self.d_model, knowledge_embed_size * self.n_heads, bias=False)
        
        self.W_skill_knowledge = nn.Linear(self.d_model, knowledge_embed_size * self.n_heads, bias=False)
        
#         self.similar = nn.CosineSimilarity(dim=0, eps=1e-6)
                
        self.softmax = nn.Softmax(dim=0)
        
        self.drop = nn.Dropout(p=0.5)
        
                # initialization
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
        
    def forward(self, batch_stu_id, batch_item_id, batch_knowledge_id):
        
        # three embedding representation in paper: [batch_size, knowledge_embed_size * n_heads]
        embed_stu = torch.sigmoid(self.emb_stu(batch_stu_id))   
        embed_item = torch.sigmoid(self.emb_item(batch_item_id))     
        embed_knowledge = torch.sigmoid(self.emb_knowledge(batch_knowledge_id)) 
        
        # three relation attention in paper: [batch_size, knowledge_embed_size * n_heads]
        stu_knowledge_attention = self.W_stu_knowledge(embed_stu)
        item_knowledge_attention = self.W_item_knowledge(embed_item)
        skill_knowledge_attention = self.W_skill_knowledge(embed_knowledge)
        
        
        attention_score = (stu_knowledge_attention * item_knowledge_attention) / np.sqrt(self.knowledge_embed_size)\
                          * skill_knowledge_attention
        
        return attention_score


class ACDM(nn.Module):
    
    def __init__(self, student_n, item_n, knowledge_n, knowledge_embed_size, n_heads=8):
        
        super(ACDM, self).__init__()
        
        self.student_n = student_n
        self.item_n = item_n
        self.knowledge_n = knowledge_n
        self.knowledge_embed_size = knowledge_embed_size
        self.n_heads = n_heads
        
        self.muti_attention = AttentionLayer(student_n, item_n, knowledge_n, knowledge_embed_size)
        
        self.similar = nn.CosineSimilarity(dim=0, eps=1e-6)
        self.softmax = nn.Softmax(dim=0)
        
        self.linear1 = nn.Linear(self.knowledge_embed_size * self.n_heads, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 1)
        
        self.drop = nn.Dropout(p=0.5)
        
                # initialization
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
        
    def forward(self, batch_stu_id, batch_item_id, batch_knowledge_id, knowledge_n):
        
        attention_score = self.muti_attention(batch_stu_id, batch_item_id, batch_knowledge_id)
        # [batch_size, ]
        hidden1 = self.drop(torch.sigmoid(self.linear1(attention_score))) 
        hidden2 = self.drop(torch.sigmoid(self.linear2(hidden1))) 
        out = torch.sigmoid(self.linear3(hidden2))
        out = out
        
        return out
    
        
    def apply_clipper(self):
        clipper = NoneNegClipper()
        self.linear1.apply(clipper)
        self.linear2.apply(clipper)
        self.linear3.apply(clipper)

In [ ]:
class MetaLearner(object):
    
    def __init__(self, model_type, train_data, val_data, test_data, \
                 student_n, item_n, knowledge_n, loss_func, \
                 knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
                 batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available = True):
        
        super(MetaLearner, self).__init__()
        
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.student_n = student_n
        self.item_n = item_n
        self.knowledge_n = knowledge_n
        self.knowledge_embed_size = knowledge_embed_size
        
        self.train_epochs = epoch_size
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model_type = model_type
        self.gpu_available = gpu_available

        self.model = self.new_model()

        # gpu
        # if self.gpu_available:
        #     self.model = self.model.to(device)
            
        self.loss_func = loss_func
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        
        # meta-leaner hyperparameters
        self.meta = False
        self.num_tasks = 100
        self.num_samples = 1
        self.task_epochs = 100
        self.alpha = 1e-3
        self.beta = 1e-3
        self.lam = 1e-3

        
        self.train_losses = []
        self.val_losses = []
        self.test_losses = []
    

    def new_model(self):
        if self.model_type == 'AGCDM':
            model = AGCDM(student_n, item_n, knowledge_n, knowledge_embed_size)
        elif self.model_type == 'ACDM':
            model = ACDM(student_n, item_n, knowledge_n, knowledge_embed_size)
        elif self.model_type == 'NeuralCDM':
            model = NeuralCDM(student_n, item_n, knowledge_n, knowledge_embed_size)
        else:
            raise ValueError('No models')

        if self.gpu_available:
            model = model.to(device)
        return model
        
    def sample_task_data(self, data):#mate过程中用到的随机抽样，抽出一个小task去fit数据集
        dataloader = Data.DataLoader(MyDataset(data), batch_size=self.num_samples, shuffle=True, num_workers=0) # dataloader本身有shuffle后的sample功能
        task_data = next(iter(dataloader))
        return task_data
    
    def show_params_grad(self):
        for params in self.model.parameters():
            print(params.grad)
            break
        
        
    def train_task(self, task_data):
        stu, item, knowledge, label = task_data[0], task_data[1], task_data[2], task_data[3]
        if self.gpu_available:
            stu, item, knowledge, label = \
            stu.to(device), item.to(device), knowledge.to(device), label.to(device)
        self.optimizer.zero_grad()
        output_1 = self.model(stu, item, knowledge, knowledge_n)
        output_0 = torch.ones(output_1.size()).to(device) - output_1
        #print(output_1.shape, output_0.shape)
        out = torch.cat((output_0, output_1), 1)
        loss_task = self.loss_func(out, label)
        loss_task.backward() 
        self.optimizer.step() 
        
    def reset_model(self):
        self.model = self.new_model()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.meta = False
        del self.train_losses[:]
        del self.val_losses[:]
        del self.test_losses[:]
        
    def learn_algorithm(self):
        
        print("Learning an algorithm from current dataset....")
        self.meta = True
        
        for e in range(self.task_epochs):        
            
            self.opti_params_ = []

            #1. for train task i in batch of tasks
            for i in range(self.num_tasks):
                
                task_data = self.sample_task_data(self.train_data)
    
                self.train_task(task_data)
                
                opti_params = deepcopy(self.model.state_dict())
                
                self.opti_params_.append(opti_params)
            
                
            meta_grad_dict = deepcopy(self.model.state_dict())
            meta_grad_dict = {name: nn.init.constant_(meta_grad_dict[name], 0.) for name in meta_grad_dict} 
            
            
            #2. Add each tasks loss, backprogate to get a "fitness" parameters
            for i in range(self.num_tasks):
                
                task_data = self.sample_task_data(train_data)
                stu, item, knowledge, label = task_data[0], task_data[1], task_data[2], task_data[3]

                if self.gpu_available:
                    stu, item, knowledge, label = \
                    stu.to(device), item.to(device), knowledge.to(device), label.to(device)
                
                net_optim = self.new_model()
                
                net_optim.load_state_dict(self.opti_params_[i])
                
                output_1 = net_optim(stu, item, knowledge, knowledge_n)
                output_0 = torch.ones(output_1.size()).to(device) - output_1
                out = torch.cat((output_0, output_1), 1)
                
                loss = self.loss_func(out, label)
                
                loss.backward()
                
                #update meta gradient bt net_optim_params's grad
                net_optim_params_grad = {}
                for name, params in zip(net_optim.state_dict(), net_optim.parameters()):
                    net_optim_params_grad[name] = params.grad.data
                #print(net_optim_params_grad)
                meta_grad_dict = {name: meta_grad_dict[name] + net_optim_params_grad[name] / self.num_samples for name in meta_grad_dict} 
                #meta_grad_dict = {name: meta_grad_dict[name] + net_optim_params[name].grad.data / self.num_samples for name in meta_grad_dict} 
            
            
            #update net params by meta gradient
            net_params = self.model.state_dict()
            net_params_new = {name: net_params[name] + self.beta * meta_grad_dict[name] / self.num_samples for name in net_params} 
            self.model.load_state_dict(net_params_new)
    
    
    def evaluate(self, data):
        self.model.eval() # 抽离
        error = 0.
        with torch.no_grad():
            dataset = MyDataset(data)
            dataloader = Data.DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=0)
            for batch_stu_id, batch_item_id, batch_knowledge_id, batch_label in dataloader:
                # gpu
                if self.gpu_available:
                    batch_stu_id, batch_item_id, batch_knowledge_id, batch_label = \
                    batch_stu_id.to(device), batch_item_id.to(device), batch_knowledge_id.to(device), batch_label.to(device)
                
                #predict = self.model(batch_stu_id, batch_item_id, batch_knowledge_id, knowledge_n)
                output_1 = self.model(batch_stu_id, batch_item_id, batch_knowledge_id, knowledge_n)
                output_0 = torch.ones(output_1.size()).to(device) - output_1
                #print(output_1.shape, output_0.shape)
                batch_out = torch.cat((output_0, output_1), 1)
                batch_error = self.loss_func(batch_out, batch_label)
                error += batch_error #/ len(data)
        self.model.train()
        return error.item()
    
        
    def train(self):
        #warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. 
        
        scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, 0.5) 
        train_dataset = MyDataset(self.train_data)
        dataloader = Data.DataLoader(train_dataset, batch_size = self.batch_size, shuffle=True, num_workers=0)
        
        
        for epoch in range(self.train_epochs):
            loss_epoch = 0.
            for batch_stu_id, batch_item_id, batch_knowledge_id, batch_label in dataloader:
                self.optimizer.zero_grad()
                #batch_out = self.model(batch_stu_id, batch_item_id, batch_knowledge_id, knowledge_n)
                                # gpu
                if self.gpu_available:
                    batch_stu_id, batch_item_id, batch_knowledge_id, batch_label = \
                    batch_stu_id.to(device), batch_item_id.to(device), batch_knowledge_id.to(device), batch_label.to(device)
                
                output_1 = self.model(batch_stu_id, batch_item_id, batch_knowledge_id, knowledge_n)
                output_0 = torch.ones(output_1.size()).to(device) - output_1

                batch_out = torch.cat((output_0, output_1), 1)
                loss_batch = self.loss_func(batch_out, batch_label)
                loss_batch.backward()
                loss_epoch += loss_batch
                self.optimizer.step()
            #loss_epoch = loss_epoch / len(self.train_data)
            self.train_losses.append(loss_epoch.item())    

            # test on validation data
            val_loss = self.evaluate(self.val_data)
            self.val_losses.append(val_loss)
            
            if self.meta == True and (val_loss - min(self.val_losses)) > 1e-1:
                break
            
            MODEL_PATH = './results/models/Experiment1/FrcSub_'

            if len(self.val_losses) == 0 or val_loss < min(self.val_losses):
                if self.meta == False:
                    torch.save(self.model.state_dict(), './results/models/Experiment1/FrcSub/'+self.model_type+'.pt')
                else:
                    torch.save(self.model.state_dict(), './results/models/Experiment1/FrcSub/Meta_'+self.model_type+'.pt')
            else:
                scheduler.step()
                self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
             
            #print("epoch: ", epoch+1, "| loss: ", loss_epoch.data.item())
            
    def binary_classify(self, data):
        data[data <= 0.5] = 0
        data[data > 0.5] = 1
        return data.astype(np.int64)
    
    def get_scores(self, true_scores, pred_scores):

#         fpr, tpr, thresholds = metrics.roc_curve(true_scores, pred_scores)
        true_scores = self.binary_classify(true_scores)
        pred_scores = self.binary_classify(pred_scores)
    
#         loss_func = nn.MSELoss()
#         rmse = np.sqrt(((true_scores - pred_scores) ** 2).mean())
        accuracy = accuracy_score(true_scores, pred_scores)
        precision = precision_score(true_scores, pred_scores)
        recall = recall_score(true_scores, pred_scores)
        f1 = f1_score(true_scores, pred_scores)
        roc_auc = roc_auc_score(true_scores, pred_scores)

        return accuracy, precision, recall, f1, roc_auc
    
    def get_test_score(self, data):
        self.model.eval() 
        error = 0.
        with torch.no_grad():
            dataset = MyDataset(data)
            dataloader = iter(Data.DataLoader(dataset, batch_size=len(data), shuffle=False, num_workers=0))
            stu_id, item_id, knowledge_id, true_scores = next(dataloader)
            #gpu
            if self.gpu_available:
                stu_id, item_id, knowledge_id, true_scores = \
                stu_id.to(device), item_id.to(device), knowledge_id.to(device), true_scores.to(device)

            true_scores = true_scores.view(-1).cpu().detach().numpy()
            
            # pred_scores = self.model(stu_id, item_id, knowledge_id, knowledge_n).view(-1).cpu().detach().numpy()
            output_1 = self.model(stu_id, item_id, knowledge_id, knowledge_n).cpu()
            output_0 = torch.ones(output_1.size()) - output_1
            batch_out = torch.cat((output_0, output_1), 1)
            pred = torch.nn.Softmax(dim=1)(batch_out)
            pred_scores = torch.argmax(pred, dim=1).detach().numpy()

            #print(true_scores.shape, pred_scores.shape) same
            # output_1 = self.model(batch_stu_id, batch_item_id, batch_knowledge_id, knowledge_n)
            # output_0 = torch.ones(output_1.size()).to(device) - output_1
            # batch_out = torch.cat((output_0, output_1), 1)
            #print(true_scores.shape, pred_scores.shape)
            accuracy, precision, recall, f1, roc_auc = self.get_scores(true_scores, pred_scores)
        self.model.train()
        return accuracy, precision, recall, f1, roc_auc
    
    def show_train_val(self, dataname='FrcSub'):
        fig, (ax1, ax2) = plt.subplots(2, 1)

        x_loss = range(len(self.train_losses))
        ax1.plot(x_loss, self.train_losses, label='train loss', color = 'g', linewidth=2)
        ax1.set_xlabel('epoch')
        ax1.set_ylabel('loss')
        #ax1.set_facecolor('lightsteelblue')
        ax1.grid(b=True, color='gray', linestyle='--', linewidth=1, alpha=0.8)
        ax1.legend()

        x_rmse = range(len(self.val_losses))
        ax2.plot(x_rmse, self.val_losses, label='val loss', color = 'r', linewidth=2)
        ax2.set_xlabel('epoch')
        ax2.set_ylabel('error')
        ax2.grid(b=True, color='gray', linestyle='--', linewidth=1, alpha=0.8)
        ax2.legend()

In [ ]:
#hyper parameters
NUM_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
EMBEDDING_SIZE = 32 #knowledge_embedding_size, dimention of knowledge space
TRAIN_NUM = 1

In [ ]:
for i in range(TRAIN_NUM):

    print('Repeat', (i+1), '>')

    FrcSub = load_data(path='./FrcSub/', val_ratio=0.8, test_ratio=0.8)
    train_data, val_data, test_data = FrcSub['train_data'], FrcSub['val_data'], FrcSub['test_data']
    student_n, item_n, knowledge_n, knowledge_embed_size = FrcSub['students_num'], FrcSub['items_num'], FrcSub['skills_num'], EMBEDDING_SIZE
    
    # results1 = collections.OrderedDict()
    # results1['loss'], results1['acc'], results1['f1'], results1['recall'], results1['auc'] = [], [], [], [], []
    # loss_func = nn.CrossEntropyLoss()
    # meta_learner = MetaLearner('NeuralCDM', train_data, val_data, test_data, \
    #                 student_n, item_n, knowledge_n, loss_func, \
    #                 knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
    #                 batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

    # meta_learner.reset_model()
    # meta_learner.train()
    # #meta_learner.show_train_val()
    # loss = meta_learner.evaluate(test_data)
    # # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

    # accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

    # print("NeuralCDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
    #       .format(loss, accuracy, precision, recall, f1, roc_auc))
    
    # results1['loss'].append(loss)
    # results1['acc'].append(accuracy)
    # results1['recall'].append(recall)
    # results1['f1'].append(f1)
    # results1['auc'].append(roc_auc)

    # # ACDM
    # results2 = collections.OrderedDict()
    # results2['loss'], results2['acc'], results2['f1'], results2['recall'], results2['auc'] = [], [], [], [], []
    # loss_func = nn.CrossEntropyLoss()
    # meta_learner = MetaLearner('ACDM', train_data, val_data, test_data, \
    #                 student_n, item_n, knowledge_n, loss_func, \
    #                 knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
    #                 batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

    # meta_learner.reset_model()
    # meta_learner.train()
    # #meta_learner.show_train_val()
    # loss = meta_learner.evaluate(test_data)
    # # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

    # accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

    # print("ACDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
    #       .format(loss, accuracy, precision, recall, f1, roc_auc))
    
    # results2['loss'].append(loss)
    # results2['acc'].append(accuracy)
    # results2['recall'].append(recall)
    # results2['f1'].append(f1)
    # results2['auc'].append(roc_auc)

    # AGCDM
    results3 = collections.OrderedDict()
    results3['loss'], results3['acc'], results3['f1'], results3['recall'], results3['auc'] = [], [], [], [], []
    loss_func = nn.CrossEntropyLoss()
    meta_learner = MetaLearner('AGCDM', train_data, val_data, test_data, \
                    student_n, item_n, knowledge_n, loss_func, \
                    knowledge_embed_size=EMBEDDING_SIZE, epoch_size=50, \
                    batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

    meta_learner.reset_model()
    meta_learner.train()
    meta_learner.show_train_val()
    loss = meta_learner.evaluate(test_data)
    # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

    accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

    print("AGCDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
          .format(loss, accuracy, precision, recall, f1, roc_auc))
    
    results3['loss'].append(loss)
    results3['acc'].append(accuracy)
    results3['recall'].append(recall)
    results3['f1'].append(f1)
    results3['auc'].append(roc_auc)

    
# torch.save(results1, './results/scores/Experiment1_NeuralCDM_' + 'FrcSub' + '.pt')
# torch.save(results2, './results/scores/Experiment1_ACDM_' + 'FrcSub' + '.pt')
# torch.save(results3, './results/scores/Experiment1_AGCDM_' + 'FrcSub' + '.pt')

In [ ]:
#results
results = collections.OrderedDict()
results['loss'], results['acc'], results['f1'], results['recall'], results['auc'] = [], [], [], [], []
for i in range(TRAIN_NUM):
    # NeuralCDM' Train with meta
    FrcSub = load_data(path='./FrcSub/', val_ratio=0.8, test_ratio=0.8)
    train_data, val_data, test_data = FrcSub['train_data'], FrcSub['val_data'], FrcSub['test_data']
    student_n, item_n, knowledge_n, knowledge_embed_size = FrcSub['students_num'], FrcSub['items_num'], FrcSub['skills_num'], EMBEDDING_SIZE
    
    loss_func = nn.CrossEntropyLoss()
    meta_learner = MetaLearner('NeuralCDM', train_data, val_data, test_data, \
                    student_n, item_n, knowledge_n, loss_func, \
                    knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
                    batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

    meta_learner.reset_model()
    meta_learner.train()
    #meta_learner.show_train_val()
    loss = meta_learner.evaluate(test_data)
    # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

    accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

    print("NeuralCDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
          .format(loss, accuracy, precision, recall, f1, roc_auc))
    
    results['loss'].append(loss)
    results['acc'].append(accuracy)
    results['recall'].append(recall)
    results['f1'].append(f1)
    results['auc'].append(roc_auc)
    
torch.save(results, './results/scores/Experiment1_NeuralCDM_' + 'FrcSub' + '.pt')

In [ ]:
#results
results = collections.OrderedDict()
results['loss'], results['acc'], results['f1'], results['recall'], results['auc'] = [], [], [], [], []
for i in range(TRAIN_NUM):
    # ACDM Train with meta
    FrcSub = load_data(path='./FrcSub/', val_ratio=0.8, test_ratio=0.8)
    train_data, val_data, test_data = FrcSub['train_data'], FrcSub['val_data'], FrcSub['test_data']
    student_n, item_n, knowledge_n, knowledge_embed_size = FrcSub['students_num'], FrcSub['items_num'], FrcSub['skills_num'], EMBEDDING_SIZE
    
    loss_func = nn.CrossEntropyLoss()
    meta_learner = MetaLearner('ACDM', train_data, val_data, test_data, \
                    student_n, item_n, knowledge_n, loss_func, \
                    knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
                    batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

    meta_learner.reset_model()
    meta_learner.train()
    #meta_learner.show_train_val()
    loss = meta_learner.evaluate(test_data)
    # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

    accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

    print("ACDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
          .format(loss, accuracy, precision, recall, f1, roc_auc))
    
    results['loss'].append(loss)
    results['acc'].append(accuracy)
    results['recall'].append(recall)
    results['f1'].append(f1)
    results['auc'].append(roc_auc)

torch.save(results, './results/scores/Experiment1_ACDM_' + 'FrcSub' + '.pt')

In [ ]:
# NUM_EPOCHS = 100  # Math1 调小epoch，ACGDM会升很高，模型较大，调高的话一次性更新的grad太多，容易过拟合
# BATCH_SIZE = 128

In [ ]:
#results
results = collections.OrderedDict()
results['loss'], results['acc'], results['f1'], results['recall'], results['auc'] = [], [], [], [], []
for i in range(TRAIN_NUM):
    # AGCDM Train with meta
    FrcSub = load_data(path='./FrcSub/', val_ratio=0.8, test_ratio=0.8)
    train_data, val_data, test_data = FrcSub['train_data'], FrcSub['val_data'], FrcSub['test_data']
    student_n, item_n, knowledge_n, knowledge_embed_size = FrcSub['students_num'], FrcSub['items_num'], FrcSub['skills_num'], EMBEDDING_SIZE
    
    loss_func = nn.CrossEntropyLoss()
    meta_learner = MetaLearner('AGCDM', train_data, val_data, test_data, \
                    student_n, item_n, knowledge_n, loss_func, \
                    knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
                    batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

    meta_learner.reset_model()
    meta_learner.train()
    #meta_learner.show_train_val()
    loss = meta_learner.evaluate(test_data)
    # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

    accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

    print("AGCDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
          .format(loss, accuracy, precision, recall, f1, roc_auc))
    
    results['loss'].append(loss)
    results['acc'].append(accuracy)
    results['recall'].append(recall)
    results['f1'].append(f1)
    results['auc'].append(roc_auc)
    
torch.save(results, './results/scores/Experiment1_AGCDM_' + 'FrcSub' + '.pt')

In [ ]:
# epoch_ls = [10, 20, 50, 100]
# batch_ls = [32, 64, 128, 256, 512]
# TRAIN_NUM = 3

In [ ]:
# #results
# for NUM_EPOCHS in epoch_ls:

#     for BATCH_SIZE in batch_ls:
#         results = collections.OrderedDict()
#         results['loss'], results['acc'], results['f1'], results['recall'], results['auc'] = [], [], [], [], []
#         for i in range(TRAIN_NUM):
#             # AGCDM Train with meta
#             FrcSub = load_data(path='./FrcSub/', val_ratio=0.8, test_ratio=0.8)
#             train_data, val_data, test_data = FrcSub['train_data'], FrcSub['val_data'], FrcSub['test_data']
#             student_n, item_n, knowledge_n, knowledge_embed_size = FrcSub['students_num'], FrcSub['items_num'], FrcSub['skills_num'], EMBEDDING_SIZE
            
#             loss_func = nn.CrossEntropyLoss()
#             meta_learner = MetaLearner('AGCDM', train_data, val_data, test_data, \
#                             student_n, item_n, knowledge_n, loss_func, \
#                             knowledge_embed_size=EMBEDDING_SIZE, epoch_size=NUM_EPOCHS, \
#                             batch_size=BATCH_SIZE, learning_rate = LEARNING_RATE, gpu_available=gpu_available)

#             meta_learner.reset_model()
#             meta_learner.train()
#             #meta_learner.show_train_val()
#             loss = meta_learner.evaluate(test_data)
#             # print("error on test data: {}".format(meta_learner.evaluate(test_data)))

#             accuracy, precision, recall, f1, roc_auc = meta_learner.get_test_score(test_data)

#             # print("AGCDM | Rmse: {:4.6f} | Accuracy: {:4.6f} | Precision: {:4.6f} | Recall: {:4.6f} | F1: {:4.6f} | AUC: {:4.6f}"\
#             #       .format(loss, accuracy, precision, recall, f1, roc_auc))
            
#             if roc_auc > 0.83:
#                 print(NUM_EPOCHS, BATCH_SIZE)
            
#             # results['loss'].append(loss)
#             # results['acc'].append(accuracy)
#             # results['recall'].append(recall)
#             # results['f1'].append(f1)
#             # results['auc'].append(roc_auc)
            
#         # torch.save(results, './results/scores/Experiment1_AGCDM_' + 'FrcSub' + '.pt')
# print('end')